In [1]:
import csv
import io
import psycopg2
import time
from functools import wraps
from memory_profiler import memory_usage
from typing import Iterator, Dict, Any, Optional

In [2]:
def profile(fn):
    @wraps(fn)
    def inner(*args, **kwargs):
        fn_kwargs_str = ', '.join(f'{k}={v}' for k, v in kwargs.items())
        print(f'\n{fn.__name__}({fn_kwargs_str})')

        # Measure time
        t = time.perf_counter()
        retval = fn(*args, **kwargs)
        elapsed = time.perf_counter() - t
        print(f'Time   {elapsed:0.4}')

        # Measure memory
        mem, retval = memory_usage((fn, args, kwargs), retval=True, timeout=200, interval=1e-7)

        print(f'Memory {max(mem) - min(mem)}')
        return retval

    return inner

In [3]:
connection = psycopg2.connect(
    host="localhost",
    database="estoque_teste",
    user="rg3915",
    password="1234",
)
connection.autocommit = True

In [4]:
def csv_to_list(filename: str) -> list:
    '''
    Lê um csv e retorna um OrderedDict.
    Créditos para Rafael Henrique
    https://bit.ly / 2FLDHsH
    '''
    with open(filename) as csv_file:
        reader = csv.DictReader(csv_file, delimiter=',')
        csv_data = [line for line in reader]
    return csv_data

In [5]:
items = csv_to_list('/tmp/produtos_1000.csv')

In [6]:
items[:5]

[{'produto': 'uUgiUtujdZCY', 'quantidade': '4404'},
 {'produto': 'VQZRSikwSBaI', 'quantidade': '383'},
 {'produto': 'OCycVVPdsHRA', 'quantidade': '9338'},
 {'produto': 'GZIXDUtvOzyT', 'quantidade': '6945'},
 {'produto': 'QkfAVbmHLPSt', 'quantidade': '5075'}]

In [7]:
@profile
def insert_one_by_one(connection, items: Iterator[Dict[str, Any]]) -> None:
    with connection.cursor() as cursor:
        for item in items:
            cursor.execute("""
                INSERT INTO core_product (title, quantity)
                VALUES (
                    %(title)s,
                    %(quantity)s
                );
            """, {
                'title': item['produto'],
                'quantity': int(item['quantidade']),
            })

In [8]:
insert_one_by_one(connection, items)


insert_one_by_one()
Time   1.276
Memory 0.1328125


In [9]:
@profile
def insert_executemany(connection, items: Iterator[Dict[str, Any]]) -> None:
    with connection.cursor() as cursor:
        all_items = [{
            'title': item['produto'],
            'quantity': int(item['quantidade'])
        } for item in items]

        cursor.executemany("""
            INSERT INTO core_product (title, quantity)
            VALUES (
                %(title)s,
                %(quantity)s
            );
        """, all_items)

In [10]:
insert_executemany(connection, items)


insert_executemany()
Time   1.511
Memory 0.015625


In [11]:
@profile
def insert_with_copy_from(connection):
    with open('/tmp/produtos_1000.csv', 'r') as f:
        next(f)
        connection.cursor().copy_from(f, 'core_product', sep=',', columns=('title', 'quantity'))

In [12]:
insert_with_copy_from(connection)


insert_with_copy_from()
Time   0.009007
Memory 0.01171875
